In [3]:
from itertools import groupby

import pandas as pd
df = pd.read_json("fashion_boutique_dataset.json")
df.head(30)

,product_id,category,brand,season,size,color,original_price,markdown_percentage,current_price,purchase_date,stock_quantity,customer_rating,is_returned,return_reason
0,FB000001,Outerwear,Zara,Spring,XL,Red,196.01,0.0,196.01,2025-07-05T00:00:00.000,37,3.0,False,None
1,FB000002,Tops,Uniqlo,Winter,L,Pink,119.64,0.0,119.64,2025-08-06T00:00:00.000,2,2.5,False,None
2,FB000003,Accessories,Uniqlo,Winter,None,Black,33.80,0.0,33.80,2025-08-06T00:00:00.000,22,4.3,False,None
3,FB000004,Shoes,Uniqlo,Spring,XL,Black,75.36,0.0,75.36,2025-07-07T00:00:00.000,48,2.6,False,None
4,FB000005,Tops,Banana Republic,Winter,XL,Black,105.02,0.0,105.02,2025-08-06T00:00:00.000,10,NaN,False,None
5,FB000006,Accessories,Zara,Fall,None,White,35.03,35.4,22.63,2025-08-06T00:00:00.000,38,2.9,True,Color Mismatch
6,FB000007,Bottoms,Zara,Spring,S,Gray,53.83,54.5,24.49,2024-10-29T00:00:00.000,38,2.6,False,None
7,FB000008,Dresses,Mango,Winter,XL,Blue,91.55,22.1,71.32,2025-08-06T00:00:00.000,47,1.6,False,None
8,FB000009,Outerwear,Uniqlo,Spring,XL,Green,227.73,12.8,198.58,2025-03-24T00:00:00.000,14,NaN,False,None
9,FB000010,Outerwear,H&M,Fall,XXL,Pink,248.41,10.9,221.33,2025-08-06T00:00:00.000,37,NaN,False,None


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2176 entries, 0 to 2175
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   product_id           2176 non-null   object 
 1   category             2176 non-null   object 
 2   brand                2176 non-null   object 
 3   season               2176 non-null   object 
 4   size                 1685 non-null   object 
 5   color                2176 non-null   object 
 6   original_price       2176 non-null   float64
 7   markdown_percentage  2176 non-null   float64
 8   current_price        2176 non-null   float64
 9   purchase_date        2176 non-null   object 
 10  stock_quantity       2176 non-null   int64  
 11  customer_rating      1814 non-null   float64
 12  is_returned          2176 non-null   bool   
 13  return_reason        320 non-null    object 
dtypes: bool(1), float64(4), int64(1), object(8)
memory usage: 223.3+ KB


In [18]:
#Mencari data size pada uniqlo karena null, siapa tau bisa dijadikan acuan jika dilihat dari harga, colour, dan season

df.loc[
    (df['category'] == 'Accessories') & (df['brand'] == 'Uniqlo') & (df['season'] == 'Winter') & (df['color'] == 'Black'),
    ['category', 'brand', 'season', 'size', 'color']
]

,category,brand,season,size,color
2,Accessories,Uniqlo,Winter,None,Black
604,Accessories,Uniqlo,Winter,None,Black
829,Accessories,Uniqlo,Winter,None,Black
1075,Accessories,Uniqlo,Winter,None,Black
1413,Accessories,Uniqlo,Winter,None,Black
1459,Accessories,Uniqlo,Winter,None,Black
1544,Accessories,Uniqlo,Winter,None,Black
1790,Accessories,Uniqlo,Winter,None,Black
1798,Accessories,Uniqlo,Winter,None,Black
2133,Accessories,Uniqlo,Winter,None,Black


In [23]:
df.groupby('category')['current_price'].sum()

category
Accessories    17393.88
Bottoms        24079.39
Dresses        31028.35
Outerwear      48672.81
Shoes          41152.78
Tops           23719.91
Name: current_price, dtype: float64

In [25]:
df.groupby('brand')['current_price'].sum()

brand
Uniqlo    186047.12
Name: current_price, dtype: float64

In [33]:
df.groupby('brand')['customer_rating'].mean()

brand
Uniqlo    2.986604
Name: customer_rating, dtype: float64

In [34]:
df.groupby('brand')['stock_quantity'].sum()

brand
Uniqlo    54265
Name: stock_quantity, dtype: int64